In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

experiments = pd.read_parquet("results_bernoulli_moments_skewed.parquet")
from distributions import SkeGTD
rng = np.random.default_rng(0)
dist2 = SkeGTD(a=0.63, r=0.9, rng=rng)
dist3 = SkeGTD(a=0.755, r=0.9, rng=rng)
dist4 = SkeGTD(a=0.88, r=0.9, rng=rng)
def mean_f(x):
    if x=="St df=1.26 r=0.9":
        return dist2.mean()
    elif x=="St df=1.51 r=0.9":
        return dist3.mean()
    elif x=="St df=1.76 r=0.9":
        return dist4.mean()
means = experiments["distribution"].apply(lambda x: mean_f(x))
experiments["error"] = np.abs(experiments["estimate"]-means)

In [8]:
experiments

,n,method,estimate,delta,distribution,error
0,100,mean,3.528825,0.100,St df=1.26 r=0.9,2.258336
1,100,mean,4.581836,0.100,St df=1.26 r=0.9,1.205324
2,100,mean,5.269734,0.100,St df=1.26 r=0.9,0.517427
3,100,mean,3.586911,0.100,St df=1.26 r=0.9,2.200250
4,100,mean,7.790793,0.100,St df=1.26 r=0.9,2.003632
...,...,...,...,...,...,...
149995,500,tm exp,2.143650,0.001,St df=1.76 r=0.9,0.758989
149996,500,tm exp,2.173917,0.001,St df=1.76 r=0.9,0.728721
149997,500,tm exp,1.909208,0.001,St df=1.76 r=0.9,0.993430
149998,500,tm exp,1.977053,0.001,St df=1.76 r=0.9,0.925585


In [9]:
range_ns = np.unique(experiments["n"])
range_deltas = -np.sort(-np.unique(experiments["delta"]))
range_distributions = np.unique(experiments["distribution"])

# organize results
df_grouped = [ experiments[ experiments["delta"] == delta ].groupby(["n", "delta", "method", "distribution"])["error"].quantile(1-delta).reset_index() for delta in range_deltas ]
df_grouped = pd.concat(df_grouped)

df_grouped_pivot = df_grouped.pivot(index=['n', 'delta', 'distribution'], columns='method', values='error').reset_index()

relative_errors = df_grouped_pivot.copy()
est_cols = [col for col in df_grouped_pivot.columns if col not in ['n', 'delta', 'distribution']]
for est in est_cols:
    relative_errors[est] = (df_grouped_pivot[est] -  df_grouped_pivot["mean"])/df_grouped_pivot["mean"]

#df_grouped
relative_errors

method,n,delta,distribution,mean,mean atm,mean exp,mean lv,mean win,mom,mom atm,mom exp,mom lv,mom win,tm,tm atm,tm exp,tm lv,tm win
0,100,0.001,St df=1.26 r=0.9,0.0,0.000000e+00,-0.000014,0.000000e+00,-0.159943,-0.953332,-0.711796,-0.952981,-0.950922,-0.952500,-0.951639,-0.643793,-0.951878,-0.950301,-0.950820
1,100,0.001,St df=1.51 r=0.9,0.0,-2.968973e-16,-0.000422,-4.376513e-07,-0.112641,-0.894873,-0.868017,-0.907830,-0.902530,-0.908020,-0.906581,-0.840058,-0.904618,-0.900849,-0.904754
2,100,0.001,St df=1.76 r=0.9,0.0,-7.102790e-02,-0.032303,-3.728503e-03,-0.708084,-0.796022,-0.810475,-0.832177,-0.818310,-0.834905,-0.832152,-0.803235,-0.824608,-0.816424,-0.824288
3,100,0.010,St df=1.26 r=0.9,0.0,0.000000e+00,-0.139696,-1.477673e-03,-0.019419,-0.791797,-0.750878,-0.785806,-0.778689,-0.781934,-0.781225,-0.704829,-0.780030,-0.772712,-0.777024
4,100,0.010,St df=1.51 r=0.9,0.0,-2.963267e-01,-0.381376,-6.158805e-02,-0.673734,-0.693287,-0.644570,-0.683001,-0.667441,-0.683056,-0.680228,-0.633941,-0.668908,-0.653019,-0.673841
5,100,0.010,St df=1.76 r=0.9,0.0,-4.553423e-01,-0.511143,-3.105161e-01,-0.537106,-0.543984,-0.477494,-0.534997,-0.509310,-0.529831,-0.542387,-0.458463,-0.509817,-0.483721,-0.506407
6,100,0.100,St df=1.26 r=0.9,0.0,1.694558e-01,0.082363,1.280048e-01,0.094343,0.001472,0.167126,0.091291,0.135249,0.096603,0.106511,0.182401,0.124300,0.172248,0.107748
7,100,0.100,St df=1.51 r=0.9,0.0,2.304397e-01,0.114962,1.820055e-01,0.117350,-0.014523,0.232783,0.126621,0.195704,0.115281,0.111812,0.260376,0.183164,0.253494,0.125277
8,100,0.100,St df=1.76 r=0.9,0.0,2.817447e-01,0.186244,2.643416e-01,0.171982,0.013639,0.288004,0.203145,0.285045,0.178182,0.132250,0.317193,0.271650,0.360141,0.203169
9,500,0.001,St df=1.26 r=0.9,0.0,-3.756112e-03,-0.085458,-1.563391e-03,-0.575641,-0.945353,-0.825444,-0.945043,-0.942875,-0.944283,-0.945542,-0.815430,-0.944042,-0.941743,-0.944509


In [11]:
name_dict = {
    "mean": "$\\overline{X}$",
    "tm": "TM",
    "mom": "MoM",
    "win": "$1 \\wedge  t^{-1}$",
    "lv": "$(1-t^2)_+$ ",
    "exp": "$e^{-t^2}$ ",
    "atm": "$\mathbf{1}_{t < 1}$"
}

    
print()
print("\\begin{table}[t!]")
print("\t\\centering")
print("\\setlength\\tabcolsep{1.5pt}")
print("\\begin{tabular*}{0.95\\textwidth}{l@{\extracolsep{\\fill}}|ccc|ccc|ccc|ccc|ccc|ccc}")
print("& \\multicolumn{9}{c|}{\\( n = 100 \\)} & \\multicolumn{9}{c}{\\( n = 500 \\)} \\\\")
print("& \\multicolumn{3}{c}{\\( \delta = 0.1 \\)} & \multicolumn{3}{c}{\\( \delta = 0.01 \\)} & \multicolumn{3}{c|}{\\( \delta = 0.001 \\)} & \\multicolumn{3}{c}{\\( \delta = 0.1 \\)} & \multicolumn{3}{c}{\\( \delta = 0.01 \\)} & \multicolumn{3}{c}{\\( \delta = 0.001 \\)} \\\\")
print("\\( \\widehat{\\mu } \\hspace{0.2cm}\\setminus \\) \\hspace{0.2cm} \\( p \\)  & \\( 1.25 \\) & \\( 1.5 \\) & \\( 1.75 \\) & \\( 1.25 \\) & \\( 1.5 \\) & \\( 1.75 \\) & \\( 1.25 \\) & \\( 1.5 \\) & \\( 1.75 \\) & \\( 1.25 \\) & \\( 1.5 \\) & \\( 1.75 \\) & \\( 1.25 \\) & \\( 1.5 \\) & \\( 1.75 \\) & \\( 1.25 \\) & \\( 1.5 \\) & \\( 1.75 \\)  \\\\")
for est in est_cols:
    if " " in est:
        line = name_dict[est.split(" ")[1]]
        #kappa, rho = est.split(" ")
        #kappa = name_dict[kappa]
        #rho = name_dict[rho]
        #if "$" in kappa:
        #    kappa = kappa.replace("$", "")
        #else:
        #    kappa = f"\\text{{{kappa}}}"
        #rho = rho.replace("$", "")
        #line = f"$\\widehat{{\\kappa}}={kappa}, \\rho(t)={rho}$"
    else:
        line = name_dict[est]
    if " " not in est:
        print("\\hline")
    for n in range_ns:
        for delta in range_deltas:
            old_distribution = None
            for distribution in range_distributions:
                best = False
                if est in best_dicts[delta][n][distribution]:
                    best = True
                change = relative_errors[ (relative_errors.n == n) & (relative_errors.delta == delta) & (relative_errors.delta == delta) & (relative_errors.distribution == distribution)][est].values[0]
                if best:
                    line += f" & \\(\\mathbf{{{change*100:.0f}}}\\)"
                else:
                    line += f" & \\({change*100:.0f}\\)"
    line += "\\\\"
    print(line)
print("\\hline")
print("\\end{tabular*}")
print(f"\\caption{{Relative difference (in \\%) of the empirical \\( 1-\\delta  \\) confidence interval error of the indicated estimators with respect to that of the empirical mean under different heavy-tailed distributions.}}")
print(f"\\label{{tab:experiment_moments_sym}}")
print("\\end{table}")



\begin{table}[t!]
	\centering
\setlength\tabcolsep{1.5pt}
\begin{tabular*}{0.95\textwidth}{l@{\extracolsep{\fill}}|ccc|ccc|ccc|ccc|ccc|ccc}
& \multicolumn{9}{c|}{\( n = 100 \)} & \multicolumn{9}{c}{\( n = 500 \)} \\
& \multicolumn{3}{c}{\( \delta = 0.1 \)} & \multicolumn{3}{c}{\( \delta = 0.01 \)} & \multicolumn{3}{c|}{\( \delta = 0.001 \)} & \multicolumn{3}{c}{\( \delta = 0.1 \)} & \multicolumn{3}{c}{\( \delta = 0.01 \)} & \multicolumn{3}{c}{\( \delta = 0.001 \)} \\
\( \widehat{\mu } \hspace{0.2cm}\setminus \) \hspace{0.2cm} \( p \)  & \( 1.25 \) & \( 1.5 \) & \( 1.75 \) & \( 1.25 \) & \( 1.5 \) & \( 1.75 \) & \( 1.25 \) & \( 1.5 \) & \( 1.75 \) & \( 1.25 \) & \( 1.5 \) & \( 1.75 \) & \( 1.25 \) & \( 1.5 \) & \( 1.75 \) & \( 1.25 \) & \( 1.5 \) & \( 1.75 \)  \\
\hline
$\overline{X}$ & \(\mathbf{0}\) & \(\mathbf{0}\) & \(\mathbf{0}\) & \(0\) & \(0\) & \(0\) & \(0\) & \(0\) & \(0\) & \(\mathbf{0}\) & \(\mathbf{0}\) & \(\mathbf{0}\) & \(0\) & \(0\) & \(0\) & \(0\) & \(0\) & \(0\)\\
$\ma

In [13]:
filtered =relative_errors[ (relative_errors.n == n) & (relative_errors.delta == delta) & (relative_errors.distribution == distribution) & (relative_errors.c_eta == c_eta)].T[4:]


In [10]:
best_dicts = dict()

for delta in range_deltas:
    best_dicts[delta] = dict()
    for n in range_ns:
        best_dicts[delta][n]=dict()
        for distribution in range_distributions:
            filtered = relative_errors[ (relative_errors.n == n) & (relative_errors.delta == delta) & (relative_errors.distribution == distribution)].T[3:]
            best_dicts[delta][n][distribution]=list(filtered.iloc[np.argsort(np.array(filtered).flatten())[:2]].index)
                

In [6]:
filtered = relative_errors[ (relative_errors.n == n) & (relative_errors.delta == delta) & (relative_errors.distribution == distribution)].T

In [7]:
filtered

,11
method,
n,500
delta,0.001
distribution,St df=1.76
mean,0.0
mean atm,-0.838203
mean exp,-0.824991
mean lv,-0.816812
mean win,-0.847451
mom,-0.729247
